<a href="https://colab.research.google.com/github/genaiconference/Agentic_RAG_Workshop/blob/main/07_agentic_rag_components.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Agentic RAG Components

In [ ]:
!git clone https://github.com/genaiconference/Agentic_RAG_Workshop.git

Cloning into 'Agentic_RAG_Workshop'...
remote: Enumerating objects: 482, done.
remote: Counting objects: 100% (253/253), done.
remote: Compressing objects: 100% (185/185), done.
remote: Compressing objects: 100% (185/185), done.
remote: Total 482 (delta 150), reused 96 (delta 68), pack-reused 229 (from 2)
Receiving objects: 100% (482/482), 75.50 MiB | 8.68 MiB/s, done.
remote: Total 482 (delta 150), reused 96 (delta 68), pack-reused 229 (from 2)
Receiving objects: 100% (482/482), 75.50 MiB | 8.68 MiB/s, done.
Resolving deltas: 100% (216/216), done.
Resolving deltas: 100% (216/216), done.
Updating files: 100% (68/68), done.
Updating files: 100% (68/68), done.


## Setup and Installations
Install necessary libraries.

In [ ]:
!pip install -r /content/Agentic_RAG_Workshop/requirements.txt --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 4.0 MB/s eta 0:00:00
     ━━━━━

## Load Environment Variables and Initialize Clients
Load environment variables containing API keys and endpoint information

In [ ]:
import os

os.chdir("/content/Agentic_RAG_Workshop/")

from dotenv import load_dotenv
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

load_dotenv()

llm = ChatOpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
    model="gpt-4.1",
    temperature=0,
)

embeddings = OpenAIEmbeddings(
    api_key=os.getenv("OPENAI_API_KEY"),
    model="text-embedding-3-small"
)

# Agentic RAG Components

## 1. Tool Creation
Create tools to retrieve the content from the private data sources

**List of Private Data Sources**
- Leave Policy Documents
- Company Annual Reports

**Public Data Sources**
- Tavily Web Search Tool

### Create Retrivers

`create_retriever_pipeline` creates an ensemble retriever as seen in previous notebooks

In [ ]:
from multivector_utils import create_retriever_pipeline

insurance_policy_retriever = create_retriever_pipeline(
    di_results_filename="Insurance_Policy_results.pkl",
    source_file_name="Insurance_Policy",
    vector_db_name="chroma-insurance-policy",
    embeddings_model=embeddings,
    llm_model=llm,
    vectorstore_exists=True
)

leave_policy_retriever = create_retriever_pipeline(
    di_results_filename="Leave_Policy_results.pkl",
    source_file_name="Leave_Policy",
    vector_db_name="chroma-leave-policy",
    embeddings_model=embeddings,
    llm_model=llm,
    vectorstore_exists=True
)

MS23_retriever = create_retriever_pipeline(
    di_results_filename="Microsoft_2023_results.pkl",
    source_file_name="Microsoft_2023",
    vector_db_name="chroma-microsoft-2023",
    embeddings_model=embeddings,
    llm_model=llm,
    vectorstore_exists=True
)

MS24_retriever = create_retriever_pipeline(
    di_results_filename="Microsoft_2024_results.pkl",
    source_file_name="Microsoft_2024",
    vector_db_name="chroma-microsoft-2024",
    embeddings_model=embeddings,
    llm_model=llm,
    vectorstore_exists=True
)

AP23_retriever = create_retriever_pipeline(
    di_results_filename="Apple_2023_results.pkl",
    source_file_name="Apple_2023",
    vector_db_name="chroma_apple-2023",
    embeddings_model=embeddings,
    llm_model=llm,
    vectorstore_exists=True
)

AP24_retriever = create_retriever_pipeline(
    di_results_filename="Apple_2024_results.pkl",
    source_file_name="Apple_2024",
    vector_db_name="chroma_apple-2024",
    embeddings_model=embeddings,
    llm_model=llm,
    vectorstore_exists=True
)

#### Create Retrievers as Tools

`create_retriever_tool` - A helper function from LangChain that lets you wrap a retriever (like a vectorstore retriever, BM25 retriever, or hybrid retriever) into a tool that an agent can use.


**What is a Retriever Tool?**

- In LangChain, retrievers are objects that fetch relevant documents given a query.

- Agents need tools to call them (like "SearchDocs" or "GetPolicyInfo").

- create_retriever_tool makes it easy to expose a retriever as a tool for an agent.

```
create_retriever_tool(
    retriever: BaseRetriever,
    name: str,
    description: str,
) -> Tool
```

 - retriever → any retriever (e.g., vectorstore.as_retriever())

 - name → tool’s unique name that the agent can call

 - description → tells the agent when/how to use this tool

In [ ]:
from langchain.tools.retriever import create_retriever_tool

Insurance_Policy_tool = create_retriever_tool(retriever=insurance_policy_retriever,
                            name = 'Insurance_Policy_Retriever',
                            description="Use this tool to answer questions related to Health Insurance Policies of a company.")

Leave_Policy_tool = create_retriever_tool(retriever=leave_policy_retriever,
                            name = 'Leave_Policy_Retriever',
                            description="Use this tool to answer questions related to Leave Policies of a company.")

Microsoft_2023_tool = create_retriever_tool(retriever=MS23_retriever,
                            name = 'Microsoft_2023_Retriever',
                            description="Use this tool to answer questions related to the annual reports of Microsoft 2023.")

Microsoft_2024_tool = create_retriever_tool(retriever=MS24_retriever,
                            name = 'Microsoft_2024_Retriever',
                            description="Use this tool to answer questions related to the annual reports of Microsoft 2024.")

Apple_2023_tool = create_retriever_tool(retriever=AP23_retriever,
                            name = 'Apple_2023_Retriever',
                            description="Use this tool to answer questions related to the annual reports of Apple 2023.")

Apple_2024_tool = create_retriever_tool(retriever=AP24_retriever,
                            name = 'Apple_2024_Retriever',
                            description="Use this tool to answer questions related to the annual reports of Apple 2024.")

#### Create WEB Tool

In [ ]:
from langchain_tavily import TavilySearch

# Initialize Tavily Search Tool
tavily_search_tool = TavilySearch(
    max_results=5,
    topic="general",
)

## 2. Defining the Prompt Template
The prompt must have input keys:

- tools: contains descriptions and arguments for each tool.
- tool_names: contains all tool names.
- agent_scratchpad: contains previous agent actions and tool outputs as a string.

Here’s an example:

In [ ]:
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    HumanMessagePromptTemplate,
    AIMessagePromptTemplate,
    PromptTemplate,
)

template = '''Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}'''

prompt = ChatPromptTemplate.from_messages([
        ("system", template),
        MessagesPlaceholder(variable_name="conversation_history", optional=True),
        HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
        AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=['agent_scratchpad'], template='{agent_scratchpad}')),
    ])

## 3. Initializing Chat History

This creates a temporary, in-memory database for storing the agent’s conversation or state. It’s fast, but the memory disappears when the program ends.

- SqliteSaver → a helper that stores your agent’s “memory” (checkpoints) in a SQLite database.

- from_conn_string(...) → tells it where the database lives.
":memory:" → means “put the database in RAM, not on disk.”

In [ ]:
from langgraph.checkpoint.sqlite import SqliteSaver

memory = SqliteSaver.from_conn_string(":memory:")

## 4. Initializing the Agent and Agent Executor
An agent is created and executed to process user queries and generate responses based on the configured tools and memory.


#### **1. create_react_agent**

We use `create_react_agent` to create a reasoning-based agent.

A factory function that creates a **ReAct-style agent**.

**ReAct** = **Reason** + **Act** → the agent alternates between:

  - **Reasoning** (thinking in natural language)

  - **Acting** (calling tools like retrievers, calculators, APIs, etc.)

**Signature (simplified):**

```
create_react_agent(llm, tools, prompt) -> Agent

```

- llm → the LLM powering reasoning (e.g., ChatOpenAI)

- tools → list of tools (retrievers, APIs, etc.)

- prompt → (optional) custom system prompt for the agent

It returns an Agent object.


#### **2. Agent Executor**

The **runtime engine** that actually runs the agent.

It handles the loop:

   - Pass user input → agent

   - Agent decides whether to answer directly or call a tool

   - Execute tool → get observation

   - Pass observation back to the agent

   - Continue until final answer


**Usage:**


```
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
result = agent_executor.invoke({"input": "What is the annual leave policy?"})
```



#### ✅ Summary

`create_react_agent` → defines the agent logic (reasoning + acting)

`AgentExecutor` → runs the agent and orchestrates tool calls

In [ ]:
from langchain.agents import AgentExecutor, create_react_agent

tools = [Leave_Policy_tool, Insurance_Policy_tool, Microsoft_2023_tool, Microsoft_2024_tool, Apple_2023_tool, Apple_2024_tool, tavily_search_tool]

# Create the agent
agent = create_react_agent(llm, tools, prompt)

# Create an executor to run the agent
agent_executor = AgentExecutor(agent=agent,
                               tools=tools,
                               verbose=True,
                               stream_runnable=True,
                               handle_parsing_errors=True,
                               max_iterations=5,
                               return_intermediate_steps=True,
                              )

## lets ask some questions to see how the relevant tools get invoked based on the user query

In [ ]:
from IPython.display import Markdown

result = agent_executor.invoke({"input": "Maternity benefits"})
Markdown(result['output'])



> Entering new AgentExecutor chain...
Thought: The question is about maternity benefits, which are typically covered under a company's leave policy or insurance policy. I should check the leave policy documents for details on maternity benefits.
Action: Leave_Policy_Retriever
Action Input: maternity benefits##1.2 Sick Leave

 Objective & Eligibility  
To give associates time off to recover and recuperate in the event of illness or accidents.
All internal associates are eligible for this policy.  
### Procedures and Conditions  
☐
The company may, at its discretion, grant sick leave to an associate, based on the
nature of sickness.
☐  
In case of absence beyond 3 days, the associate is required to submit a medical
certificate.  
In case where absence is likely to be prolonged beyond 3 days, the likely duration of
☐
absence along with the doctor's certificate certifying the same should be submitted.  
[ Any sick leave in excess of 30 continuous days will require the approval of the div

Maternity benefits are provided under the "Parental Leave" policy. Both birthing mothers and non-birthing parents are eligible for 26 weeks of parental leave following the birth, surrogacy, or adoption of a child. This leave must be availed within one year of the child's birth, surrogacy, or adoption. Any unused parental leave after one year will be forfeited and cannot be financially compensated or carried forward. For further details, refer to the Parental Leave Policy - India.

In [ ]:
result = agent_executor.invoke({"input": "Microsoft 2023 Profit"})
Markdown(result['output'])



> Entering new AgentExecutor chain...
Thought: To answer this, I need to look up the net profit (net income) reported by Microsoft for the fiscal year 2023.
Action: Microsoft_2023_Retriever
Action Input: net profit for 2023#PART II Item 7

effect of this change in estimate for fiscal year 2023 was an increase in operating income of $3.7 billion and net income
of $3.0 billion, or $0.40 per both basic and diluted share.

#ITEM 8. FINANCIAL STATEMENTS AND SUPPLEMENTARY DATA INCOME STATEMENTS

<table>
<tr>
<th colspan="4">(In millions, except per share amounts)</th>
</tr>
<tr>
<th>Year Ended June 30,</th>
<th>2023</th>
<th>2022</th>
<th>2021</th>
</tr>
<tr>
<td>Revenue:</td>
<td></td>
<td></td>
<td></td>
</tr>
<tr>
<td>Product</td>
<td>$ 64,699</td>
<td>$ 72,732</td>
<td>$ 71,074</td>
</tr>
<tr>
<td>Service and other</td>
<td>147,216</td>
<td>125,538</td>
<td>97,014</td>
</tr>
<tr>
<td>Total revenue</td>
<td>211,915</td>
<td>198,270</td>
<td>168,088</td>
</tr>
<tr>
<td>Cost of revenue:</

Microsoft's net profit (net income) for the fiscal year 2023 was $72,361 million (or $72.36 billion).